In [1]:
import pyspark
from pyspark.sql import SparkSession
import os

## DEFINE SENSITIVE VARIABLES
NESSIE_URI = "http://nessie:19120/api/v1"
MINIO_HOST = 'http://minio:9000'
MINIO_ACCESS_KEY = "jeAncqX06SoMwYqb3boy"
MINIO_SECRET_KEY = "jR2iwRbXnSpQOj5Yg9Zxw1kNGOtJ1UvUKbXcqLGT"

jar_packages = [
  "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1",
  "org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.95.0",
  "software.amazon.awssdk:bundle:2.17.178",
  "software.amazon.awssdk:url-connection-client:2.17.178"
]

spark_extensions = [
  "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
  "org.projectnessie.spark.extensions.NessieSparkSessionExtensions"
]

# fs.s3a.path.style.access true
# dremio.s3.compat true
# fs.s3a.endpoint http://minio:9000

conf = (
  pyspark.SparkConf()
    .setAppName('iceberg_hello_world')
    .set('spark.jars.packages', ','.join(jar_packages))
	.set('spark.sql.extensions', ','.join(spark_extensions))
	.set('spark.sql.catalog.nessie', "org.apache.iceberg.spark.SparkCatalog")
	.set('spark.sql.catalog.nessie.uri', NESSIE_URI)
	.set('spark.sql.catalog.nessie.ref', 'main')
	.set('spark.sql.catalog.nessie.authentication.type', 'NONE')
	.set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
	.set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
	.set('spark.sql.catalog.nessie.s3.endpoint', MINIO_HOST)
	.set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse')
	.set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
	.set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_KEY)
	.set('spark.hadoop.fs.s3a.path.style.access', 'true')
    .set('spark.hadoop.fs.s3a.connection.timeout', '600000')
	.set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
	.set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false')
	.set('spark.hadoop.fs.s3a.endpoint', MINIO_HOST)		
)

# .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
# .config("spark.hadoop.fs.s3a.access.key", "SEU_ACCESS_KEY") \
# .config("spark.hadoop.fs.s3a.secret.key", "SEU_SECRET_KEY") \
# .config("spark.hadoop.fs.s3a.endpoint", "http://SEU_MINIO_ENDPOINT:9000") \
# .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    
## Start Spark Session
spark = SparkSession.builder.config(conf=conf).master("local").getOrCreate()
print("Spark Running")



:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-50c4a47a-410a-4522-a9ce-7049653a9088;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.95.0 in central
	found software.amazon.awssdk#bundle;2.17.178 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.17.178 in central
	found software.amazon.awssdk#utils;2.17.178 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found softwa

Spark Running


In [2]:
data_stage = [
    (3, "Alecio", 33),
    (4, "Rosemberg", 31),
    (5, "Felipe", 35)
]
df_stage = spark.createDataFrame(data_stage, ["id", "name", "age"])
df_stage.createOrReplaceTempView("view_stage")

In [3]:
spark.sql("""USE nessie""")

spark.sql("CREATE OR REPLACE TABLE test_table (id BIGINT,name STRING, age INT)")

spark.sql("""
INSERT INTO test_table
VALUES
	(1, 'Alice', 30),
	(2, 'Bob', 25),
	(3, 'Charlie', 35)
""")


spark.sql("""SELECT * FROM test_table""").show()
spark.sql("""SELECT * FROM view_stage""").show()

++
||
++
++



SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


++
||
++
++



++
||
++
++

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 30|
|  2|    Bob| 25|
|  3|Charlie| 35|
+---+-------+---+



+---+---------+---+
| id|     name|age|
+---+---------+---+
|  3|   Alecio| 33|
|  4|Rosemberg| 31|
|  5|   Felipe| 35|
+---+---------+---+



In [4]:
spark.sql("""
MERGE INTO test_table
USING (SELECT * FROM view_stage) b
ON test_table.id = b.id
    WHEN MATCHED THEN UPDATE SET test_table.name = b.name
    WHEN NOT MATCHED THEN INSERT *
""").show()

spark.sql("""SELECT * FROM nessie.test_table""").distinct().show()

++
||
++
++

+---+---------+---+
| id|     name|age|
+---+---------+---+
|  5|   Felipe| 35|
|  3|   Alecio| 35|
|  1|    Alice| 30|
|  4|Rosemberg| 31|
|  2|      Bob| 25|
+---+---------+---+

